In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np

In [2]:
data = pd.read_csv("../master_dataset/processed_data.csv")
list(data.columns)
# drop unwanted features

# Dropped all categorical data 

data = data.drop(['title', 'text', 'text_without_stopwords', 'title_without_stopwords','syllables', 'polarity_category', 'overall_content', 'polarity_category_Neutral',
 'polarity_category_Positive'], axis=1)
#'Topic 1 Probability', 'Topic 2 Probability', 'Topic 3 Probbility' , 'Topic 4 Probability' ,'Topic 5 Probability',
#'title_word_count', 'title_sentence_count', 'title_average_word_length','title_punctuation_count', 'title_stopwords_count'  
# 'polarity_category_Neutral' , 'polarity_category_Positive'

In [3]:
#dataset is slightly imbalanced so we will perform upsampling to balance the dataset.
data['class'].value_counts()

0    21196
1    17462
Name: class, dtype: int64

In [4]:
list(data.columns)

['class',
 'text_word_count',
 'title_word_count',
 'text_sentence_count',
 'title_sentence_count',
 'text_average_word_length',
 'title_average_word_length',
 'text_punctuation_count',
 'title_punctuation_count',
 'text_stopwords_count',
 'title_stopwords_count',
 'flesch_readability',
 'subjectivity',
 'polarity',
 'Topic 1 Probability',
 'Topic 2 Probability',
 'Topic 3 Probbility',
 'Topic 4 Probability',
 'Topic 5 Probability']

In [5]:
data.describe()

,class,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000
mean,0.451705,411.374515,12.161209,15.050830,1.067877,5.138347,5.592058,53.119147,1.715635,155.440685,1.136789,0.006674,0.402753,0.056997,0.190953,0.324903,0.125233,0.185360,0.173552
std,0.497669,322.881353,3.765686,11.985707,0.270277,1.875575,1.430398,63.542364,1.452448,123.275458,1.295738,1.003826,0.124806,0.105170,0.281023,0.312565,0.210573,0.258829,0.234861
min,0.000000,1.000000,1.000000,1.000000,1.000000,2.250000,3.000000,0.000000,0.000000,0.000000,0.000000,-83.616811,0.000000,-1.000000,0.000376,0.000246,0.000120,0.000303,0.000268
25%,0.000000,220.000000,10.000000,7.000000,1.000000,4.866522,5.000000,26.000000,1.000000,80.000000,0.000000,-0.169506,0.337127,0.000000,0.002949,0.010969,0.001305,0.003113,0.003024
50%,0.000000,376.000000,11.000000,13.000000,1.000000,5.075727,5.500000,45.000000,2.000000,142.000000,1.000000,0.050549,0.405745,0.054757,0.030471,0.231605,0.006765,0.056382,0.056373
75%,1.000000,522.000000,14.000000,20.000000,1.000000,5.272987,6.066667,67.000000,3.000000,201.000000,2.000000,0.267479,0.475000,0.108144,0.282926,0.584998,0.157612,0.272065,0.268291
max,1.000000,8436.000000,45.000000,321.000000,4.000000,149.000000,149.000000,7295.000000,26.000000,3017.000000,15.000000,2.614284,1.000000,1.000000,0.997819,0.998037,0.996771,0.998911,0.998767


In [6]:
#first split the dataset into training and test sets
x = data.iloc[:,1:]
y = data.iloc[:,:1]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state = 4222)


x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.25, random_state = 4222)


#balance x_train with oversampling
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
# oversample = RandomOverSampler(sampling_strategy = 1)
x_train,y_train = undersample.fit_resample(x_train, y_train)
data = pd.concat([x_train,y_train],axis = 1)

#check that train set is oversampled
data['class'].value_counts()

0    10477
1    10477
Name: class, dtype: int64

In [7]:
x_train

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
0,425,12,16,1,5.275862,4.500000,56,3,155,3,0.133861,0.296505,-0.018548,0.498114,0.001739,0.344903,0.068380,0.086864
1,520,9,20,1,5.432220,8.222222,55,3,180,1,-0.265701,0.444944,0.085383,0.002260,0.215802,0.182273,0.598110,0.001556
2,407,16,14,1,4.704545,4.875000,45,2,158,5,0.202915,0.394314,0.019166,0.463356,0.359014,0.001191,0.112402,0.064036
3,81,11,2,1,5.256410,5.454545,8,3,29,2,-0.307371,0.062500,-0.062500,0.967485,0.008552,0.005836,0.009452,0.008675
4,129,12,4,1,4.843750,5.333333,18,3,50,2,0.144425,0.610417,-0.402083,0.987776,0.003183,0.002178,0.003533,0.003330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20949,565,16,10,1,4.742857,5.533333,46,1,256,0,-0.267928,0.368203,0.122754,0.002057,0.364870,0.000975,0.630648,0.001450
20950,1809,32,144,1,4.418500,4.875000,307,3,772,9,0.852104,0.467778,-0.001175,0.015731,0.304634,0.000364,0.000592,0.678679
20951,418,24,15,2,4.597087,4.500000,37,7,190,7,0.432620,0.487291,0.011708,0.002322,0.914441,0.062097,0.019515,0.001625
20952,677,14,25,1,4.559398,5.538462,86,2,281,0,0.490826,0.482386,0.237027,0.001734,0.258729,0.274193,0.419005,0.046338


In [8]:
# Since we will be performing SVC, and SVC works better on scaled data, we will be scaling all our to ensure that the model runs smoothly
# We will use the Min Max scaler

scaler = StandardScaler()

#
cols = data.iloc[:,0:-6].columns

# Normalize the data , since the data is not normally distributed we will use minmaxscaler
x_train[cols] = scaler.fit_transform(x_train[cols]) 

x_test[cols] = scaler.transform(x_test[cols])
x_validation[cols] = scaler.transform(x_validation[cols])



In [9]:
x_train.describe()

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,2.095400e+04,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000
mean,3.386726e-17,2.590662e-14,3.118363e-16,-3.229942e-15,3.891847e-16,-5.178109e-17,-1.028601e-15,5.049586e-15,8.689347e-17,3.298107e-14,7.403430e-17,-4.710262e-17,1.090089e-16,0.178115,0.339096,0.124739,0.179236,0.178815
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,0.272836,0.314138,0.208368,0.253806,0.236889
min,-1.268197e+00,-2.936760e+00,-1.195595e+00,-2.599167e-01,-1.583127e+00,-1.629785e+00,-7.895130e-01,-1.191022e+00,-1.261005e+00,-8.170303e-01,-8.145388e+01,-3.260156e+00,-9.843729e+00,0.000376,0.000264,0.000181,0.000303,0.000268
25%,-5.777731e-01,-6.234501e-01,-6.840047e-01,-2.599167e-01,-1.492385e-01,-4.046532e-01,-4.054829e-01,-5.021250e-01,-6.001975e-01,-8.170303e-01,-1.886156e-01,-5.295754e-01,-5.360402e-01,0.002886,0.026760,0.001305,0.002987,0.003178
50%,-1.123536e-01,-1.093813e-01,-1.724142e-01,-2.599167e-01,-3.381730e-02,-5.461558e-02,-1.248455e-01,1.867722e-01,-1.086214e-01,-5.512318e-02,4.439940e-02,2.403548e-02,-2.167820e-02,0.023639,0.259340,0.008074,0.052539,0.062548
75%,3.345723e-01,6.617219e-01,4.244414e-01,-2.599167e-01,7.723951e-02,2.954221e-01,2.148734e-01,8.756695e-01,3.668374e-01,7.067839e-01,2.726070e-01,5.787711e-01,4.752505e-01,0.248312,0.605468,0.159012,0.260281,0.280705
max,2.473057e+01,8.372754e+00,1.653954e+01,1.040515e+01,7.705982e+01,9.760589e+01,1.069605e+02,1.189803e+01,2.305186e+01,1.061158e+01,2.428522e+00,4.737978e+00,8.771649e+00,0.995947,0.998037,0.995137,0.998911,0.998767


Linear SVM

In [10]:
# Linear SVM baseline model

linearSVC = SVC(kernel='linear',random_state=4222)
linearSVC.fit(x_train, np.ravel(y_train))
y_predval_linearSVC = linearSVC.predict(x_validation)

#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_linearSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_linearSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_linearSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_linearSVC))
print("-------------------------------")


#test metrics
y_pred_linearSVC = linearSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_pred_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_pred_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_pred_linearSVC))


Performance on Validation set:
Accuracy: 0.964692188308329
Precision: 0.9690691566968194
Recall: 0.9521215596330275
F1_score: 0.9605206073752712
-------------------------------
Performance on  Test set:
Accuracy: 0.9595188825659596
Precision: 0.966315172817809
Recall: 0.9433800400343151
F1_score: 0.9547098827955434


In [11]:
from sklearn.feature_selection import RFE


# Recursive Feature Selection

svc_lin=SVC(kernel='linear',random_state=4222)
svm_rfe_model=RFE(estimator=svc_lin)
svm_rfe_model_fit=svm_rfe_model.fit(x_train,np.ravel(y_train))
feat_index = pd.Series(data = svm_rfe_model_fit.ranking_, index = x_train.columns)
signi_feat_rfe = feat_index[feat_index==1].index

print('Significant features from RFE',signi_feat_rfe)


Significant features from RFE Index(['text_word_count', 'title_word_count', 'text_sentence_count',
       'text_punctuation_count', 'text_stopwords_count',
       'title_stopwords_count', 'Topic 1 Probability', 'Topic 2 Probability',
       'Topic 4 Probability'],
      dtype='object')


In [13]:
# New Data after feature selection
x_train_new = x_train[signi_feat_rfe]
x_validation_new = x_validation[signi_feat_rfe]
x_test_new = x_test[signi_feat_rfe]



In [14]:
#hyperparameter tuning with gridsearch for SVM  

grid_params = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001]
    }

scorer = metrics.make_scorer(metrics.f1_score)

gridCV = GridSearchCV(SVC(kernel='linear',random_state = 4222), param_grid = grid_params, cv = 5, scoring = scorer, n_jobs=-1)

gridCV.fit(x_train_new,np.ravel(y_train))

print("Best Hyper Parameters: ", gridCV.best_params_)

Best Hyper Parameters:  {'C': 100, 'gamma': 1}


In [15]:
finalSVC = SVC( kernel= 'linear', C= 100, gamma= 1, random_state = 4222)
finalSVC.fit(x_train_new, np.ravel(y_train))
y_predval_finalSVC = finalSVC.predict(x_validation_new)


#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_finalSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_finalSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_finalSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_finalSVC))
print("-------------------------------")


#test metrics
y_predtest_linearSVC = finalSVC.predict(x_test_new)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_predtest_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_predtest_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_predtest_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_predtest_linearSVC))


Performance on Validation set:
Accuracy: 0.9609415416451113
Precision: 0.9647024504084014
Recall: 0.9481077981651376
F1_score: 0.9563331405436668
-------------------------------
Performance on  Test set:
Accuracy: 0.956414899120538
Precision: 0.9655274012964055
Recall: 0.9370889333714613
F1_score: 0.9510956319837468
